In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

import json
import httpx

#------------------ settings ------------------
api_key=os.getenv("DEEPSEEK_API_KEY")
base_url=os.getenv("DEEPSEEK_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'deepseek-reasoner'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

online_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = local_settings
#------------------ settings ------------------

In [4]:
llm = ChatOpenAI(model=settings['id'],api_key=settings['api_key'],base_url=settings['base_url'],temperature=0.01)

In [45]:
standard_prompt = """# 提取标准名称和标准号的提示词

## 任务描述
你是一位专业的文档分析师，负责从用户提供的中文文章或文本中提取标准名称和标准号，并以给定的json schema格式输出。目标是准确识别所有提到的标准（包括国家标准、法规、行业标准等），提取其名称（不含引号）和编号（或发布/施行日期），并确保输出清晰、格式统一，适配structure_output模式。

## 规则
请按照以下步骤处理用户提供的中文文本，并生成结构化输出：

1. **输入接收**：接收用户提供的中文文章或文本片段。
2. **提取过程**：
   - 扫描文本，识别所有提到的国家标准、行业标准、规范、法律或法规。
   - 提取每个标准的名称（去除原文中的引号，如《》）和标准号（如 GB50406-2007）或发布/施行日期（如 2015年1月1日起施行）。
   - 如果标准号和日期同时存在，优先提取标准号；如果仅有日期，则记录日期。
   - 忽略非标准相关的文本，确保只提取明确的标准信息。
3. **输出格式**：
   - 单个标准Standard对象包含的字段如下：
     - **标准名称**：标准的完整名称，不含引号（如 中华人民共和国环境保护法）。
     - **标准号/日期**：标准编号（如 GB50406-2007）或发布/施行日期（如 2015年1月1日起施行）。如果没有编号，留空，示例:""。
   - 如果文本中没有标准信息，返回对象的standards字段为空列表[]
   - 最终输出的StandardList包含了一个standards字段，数据类型为List[Standard]。
   - 输出适配structure_output模式，确保表格格式清晰，字段规范。
   - Finally output, provide a list of standards in JSON format, wrapped in a 'standards' field as specified in the schema.
4. **约束**：
   - 仅提取明确的标准化文件名称和编号/日期，排除非标准化的描述。
   - 标准名称不含原文中的引号（如《》），直接提取核心名称。
   - 如果标准号或日期格式不一致（如括号内日期或无括号），统一提取并记录原文格式。
   - 所有输出均使用中文，表格格式清晰，易于阅读。
   - 如果标准名称或编号不完整（如缺少编号），编号字段留空：""。
5. 仔细审查文档，严格禁止遗漏任何可能出现的标准信息。```"""

In [5]:
llm.invoke('hi')

AIMessage(content='\n\nHello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 9, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen3-235B', 'system_fingerprint': None, 'id': 'chatcmpl-e93a0d29b31441c4a97f3f380b48c79c', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac0d3529-0677-497d-b124-f9c0542d1a89-0', usage_metadata={'input_tokens': 9, 'output_tokens': 95, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

In [46]:
from pydantic import BaseModel, Field
from typing import List

# 定义 Pydantic 模型
class Standard(BaseModel):
    name: str = Field(description="标准|法规名称")
    code: str = Field(description="标准|法规 编号|日期")

# 定义包含 Standard 列表的 Pydantic 模型
class StandardList(BaseModel):
    standards: List[Standard] = Field(description="List of standards")

# 提取 JSON Schema
schema = Standard.model_json_schema()
print(schema)

{'properties': {'name': {'description': '标准|法规名称', 'title': 'Name', 'type': 'string'}, 'code': {'description': '标准|法规 编号|日期', 'title': 'Code', 'type': 'string'}}, 'required': ['name', 'code'], 'title': 'Standard', 'type': 'object'}


In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

# 初始化 PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=StandardList)

# 获取 JSON Schema 的格式化指令
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

Format Instructions:
 The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Standard": {"properties": {"name": {"description": "标准|法规名称", "title": "Name", "type": "string"}, "code": {"description": "标准|法规 编号|日期", "title": "Code", "type": "string"}}, "required": ["name", "code"], "title": "Standard", "type": "object"}}, "properties": {"standards": {"description": "List of standards", "items": {"$ref": "#/$defs/Standard"}, "title": "Standards", "type": "array"}}, "required": ["standards"]}
```


In [48]:
# 定义 PromptTemplate
prompt = PromptTemplate(
    template=standard_prompt+"\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

In [49]:
chain = prompt | llm | parser

In [50]:
input_data='''设计原则
烟气净化系统采取全烟气净化，不设计烟气旁路系统，以“先进技术、配置合理、造价及运行费用经济、装置实用性强”为原则，采用成熟可靠的技术装备和控制系统。整个系统的设计基本原则为：
1）以最新环保标准为准绳，面向未来，采用最佳解决方案；使烟气中的SO2，NOx，二噁英、粉尘等排放限值要优于环保部发布的“关于发布《钢铁烧结、球团工业大气污染物排放标准》等20项国家污染物排放标准修改单的公告（征求意见稿）”中烧结新标准要求，达到超净排放要求。
2）烟气净化工艺遵循技术成熟、设备先进运行稳定、操作维护方便、自动化程度较高、无二次污染原则；
3）在工艺合理、安全可靠、流程顺畅的前提下，降低运行费用，节省投资、方便施工；
4）装置能快速投入运行，适应风量及SO2、NOX负荷波动能力强；
5）烟气净化装置尽可能按现有设备状况及场地条件进行布置，力求工艺流程和设施布置合理、操作安全、简便，尽量减小项目实施时对现有单元生产的影响；
6）严格执行国家、地方及企业的有关环保、安全卫生、节能、工程设计统一技术规定等有关标准、规范。
7）执行的设计规范：
《中华人民共和国环境保护法》（2015年1月1日起施行）
《建设项目环境保护管理条例》（1998年11月29日发布施行）
《钢铁工业环境保护设计规范》（GB50406-2007）
《钢铁烧结、球团工业大气污染物排放标准》（GB28662-2012）
《建筑地基基础设计规范》（GB50007-2011）
《建筑结构荷载规范》 
《混凝土结构设计规范》
《钢结构设计规范》（GB50017-2003）
《砌体结构设计规范》（GB50003-2011）。
《建筑抗震设计规范》（GB50011-2010）。
《建筑工程抗震设防分类标准》（GB50223－2008）。
《动力机器基础设计规范》（GB50040-96）。
《烟囱设计规范》（GB50051-2013）。
《建筑桩基技术规范》（
《岩土工程勘察规范》GB 50021-2001 （2009年版）。
《通用用电设备配电设计规范》   GB50055-2011
《建筑物防雷设计规范》    
《建筑设计防火规范》       GB50016-2014 
《3～110KV高压配电装置设计规范》   GB50060-2008
《供配电系统设计规范》     GB50052-2009
《低压配电设计规范》       GB50054-2011
《20kV及以下变电所设计规程》GB50053-2013
《电力装置的继电保护和自动装置设计规范》  GB\T50062-2008
《工业企业噪声控制设计规范》(GB/T 50087-2013)
《工业企业总平面设计规范》
《工业企业设计卫生标准》（GBZ1-2010）
《工业企业煤气安全规程》 (GB6222-2005)
《起重机械安全规程》（GB6067.1-2010）
《工作场所有害因素职业接触限值第一部分：化学有害因素》（GBZ2.1-2007）
《工作场所有害因素职业接触限值第二部分：物理因素》（GBZ2.2-2007）
《建筑采光设计标准》(GB50033-2013）
《建筑照明设计标准》(GB50034-2013）
执行以及国家现行的其它规范、规程、规定。'''

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\11798\AppData\Local\Temp\ipykernel_41260\2781049100.py:1: SyntaxWarning: invalid escape sequence '\T'
  input_data='''设计原则


In [51]:
result = chain.invoke({"query": input_data})
print("Structured Output:\n", result)

Structured Output:
 standards=[Standard(name='中华人民共和国环境保护法', code='2015年1月1日起施行'), Standard(name='建设项目环境保护管理条例', code='1998年11月29日发布施行'), Standard(name='钢铁工业环境保护设计规范', code='GB50406-2007'), Standard(name='钢铁烧结、球团工业大气污染物排放标准', code='GB28662-2012'), Standard(name='建筑地基基础设计规范', code='GB50007-2011'), Standard(name='建筑结构荷载规范', code=''), Standard(name='混凝土结构设计规范', code=''), Standard(name='钢结构设计规范', code='GB50017-2003'), Standard(name='砌体结构设计规范', code='GB50003-2011'), Standard(name='建筑抗震设计规范', code='GB50011-2010'), Standard(name='建筑工程抗震设防分类标准', code='GB50223－2008'), Standard(name='动力机器基础设计规范', code='GB50040-96'), Standard(name='烟囱设计规范', code='GB50051-2013'), Standard(name='建筑桩基技术规范', code=''), Standard(name='岩土工程勘察规范', code='GB 50021-2001 （2009年版）'), Standard(name='通用用电设备配电设计规范', code='GB50055-2011'), Standard(name='建筑物防雷设计规范', code=''), Standard(name='建筑设计防火规范', code='GB50016-2014'), Standard(name='3～110KV高压配电装置设计规范', code='GB50060-2008'), Standard(name='供配电系统设计规范', code='GB50052-2009'), Sta

In [52]:
result.standards

[Standard(name='中华人民共和国环境保护法', code='2015年1月1日起施行'),
 Standard(name='建设项目环境保护管理条例', code='1998年11月29日发布施行'),
 Standard(name='钢铁工业环境保护设计规范', code='GB50406-2007'),
 Standard(name='钢铁烧结、球团工业大气污染物排放标准', code='GB28662-2012'),
 Standard(name='建筑地基基础设计规范', code='GB50007-2011'),
 Standard(name='建筑结构荷载规范', code=''),
 Standard(name='混凝土结构设计规范', code=''),
 Standard(name='钢结构设计规范', code='GB50017-2003'),
 Standard(name='砌体结构设计规范', code='GB50003-2011'),
 Standard(name='建筑抗震设计规范', code='GB50011-2010'),
 Standard(name='建筑工程抗震设防分类标准', code='GB50223－2008'),
 Standard(name='动力机器基础设计规范', code='GB50040-96'),
 Standard(name='烟囱设计规范', code='GB50051-2013'),
 Standard(name='建筑桩基技术规范', code=''),
 Standard(name='岩土工程勘察规范', code='GB 50021-2001 （2009年版）'),
 Standard(name='通用用电设备配电设计规范', code='GB50055-2011'),
 Standard(name='建筑物防雷设计规范', code=''),
 Standard(name='建筑设计防火规范', code='GB50016-2014'),
 Standard(name='3～110KV高压配电装置设计规范', code='GB50060-2008'),
 Standard(name='供配电系统设计规范', code='GB50052-2009'),
 Standard(name

In [54]:
standard_check_prompt = """你是一个专业的标准检测分析助手，任务是识别提供的国家标准（GB）和行业标准（如JB、QB等）是否过期以及有效，根据提供的标准号、标准名称，并验证标准名称与标准号是否匹配。以下是具体要求：
输入内容：接收包含标准号（如GB 12345-2018、JB/T 7890-2015等）、标准名称以及可能的发布日期文本。
识别过期标准：
检查标准号，结合你所知道的标准状态或有效性信息，判断标准是否过期。
识别标准号和名称：
识别标准号（如GB 12345-2018、JB/T 7890-2015）。
识别标准名称（如《家用电器安全标准》、《机械设备技术规范》）。
确保标准号与名称一一对应，避免混淆。
验证名称与标号匹配：
检查标准号与标准名称是否逻辑一致（例如，GB 12345-2018是否对应《家用电器安全标准》）。
如果发现标准号与名称不匹配（如GB 12345-2018对应《食品卫生标准》），标记为不匹配并说明可能的原因（如录入错误或标准号更新）。
你每次都会接收到一对标准号和名称，需要对其进行验证。
输出格式：
以清晰的结构化形式输出结果，包含以下内容：
标准号
标准名称
状态（有效/废止/待实施/无法判断）
名称与标号是否匹配（是/否）
原因（判断的依据和理由）"""

In [55]:
from enum import Enum

# 定义状态枚举
class StatusEnum(str, Enum):
    VALID = "有效"
    INVALID = "废止"
    PENDING = "待实施"
    UNKNOWN="无法判断"

class StandardCheck(BaseModel):
    name: str = Field(description="标准|法规名称")
    code: str = Field(description="标准|法规 编号|日期")
    status: StatusEnum = Field(description="状态，枚举值：有效、废止、待实施、无法判断")
    isMatch: bool = Field(description="标准号与名称是否匹配")
    reason: str = Field(description="原因")

In [56]:
# 初始化 PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=StandardCheck)

# 获取 JSON Schema 的格式化指令
format_instructions = parser.get_format_instructions()
print("Format Instructions:\n", format_instructions)

Format Instructions:
 The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"StatusEnum": {"enum": ["有效", "废止", "待实施", "无法判断"], "title": "StatusEnum", "type": "string"}}, "properties": {"name": {"description": "标准|法规名称", "title": "Name", "type": "string"}, "code": {"description": "标准|法规 编号|日期", "title": "Code", "type": "string"}, "status": {"$ref": "#/$defs/StatusEnum", "description": "状态，枚举值：有效、废止、待实施、无法判断"}, "isMatch": {"description": "标准号与名称是否匹配", "title": "Ismatch", "type": "boolean"}, "reason": {"description": "原因", "title": "Reason", "type": "string"}}, "required": ["name", "code

In [57]:
# 定义 PromptTemplate
prompt = PromptTemplate(
    template=standard_check_prompt+"\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

In [58]:
chain = prompt | llm | parser

In [67]:
input_data='name:中华人民共和国环境保护法，code:2015年1月1日起施行'

In [68]:
result = chain.invoke({"query": input_data})
print("Structured Output:\n", result)

Structured Output:
 name='中华人民共和国环境保护法' code='2015年1月1日起施行' status=<StatusEnum.UNKNOWN: '无法判断'> isMatch=False reason='code字段未提供标准号（如GB/T 1234-2015），无法验证有效性；名称与标号不匹配可能因录入错误或字段用途混淆'
